In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt
from ipyleaflet import Map, GeoJSON, GeoData, basemaps, LayersControl
from shapely.geometry import Point, Polygon, LineString
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
import rasterio
import rasterio.mask
from rasterio.plot import reshape_as_image 
from rasterio.plot import reshape_as_raster
from shapely.ops import unary_union

Import other info

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [17]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
main_db['Incoming to'] = label_encoder.fit_transform(main_db['to_berth'])
main_db


,imo,mmsi,name,from,to,from_location,to_location,from_berth,to_berth,from_haven,...,from_lat,to_long,to_lat,device_mmsi,navigation_draught,length,width,vessel_type,vessel_type_Code,Incoming to
16,9174567.0,244216000.0,VB FURIE,2022-09-06 16:24:37+00:00,2022-09-06 17:18:43.250000+00:00,"[4.29652, 51.89164]","[4.32237, 51.89391]",1E WERKH STEINWEG KADE 2,NIEUWE MAAS ALIPHOS STG,1WH,...,51.89164,4.32237,51.89391,205103000,5.8,88.0,13.0,cargo,0,190
17,9060699.0,245907000.0,TEXELBANK,2022-09-06 16:26:33+00:00,2022-09-06 17:14:54+00:00,"[4.29649, 51.89163]","[4.30626, 51.88997]",1E WERKH STEINWEG KADE 2,1E WERKH SLB,1WH,...,51.89163,4.30626,51.88997,205103000,5.8,88.0,13.0,cargo,0,11
273,9695509.0,244900124.0,VB MARS,2023-03-29 11:38:21+00:00,2023-03-29 12:29:00+00:00,"[4.18029, 51.94091]","[4.19519, 51.93326]",CALANDK HBR P 79,CALANDK PALEN 82,CKVTTI,...,51.94091,4.19519,51.93326,205770000,5.6,97.0,16.0,tanker,7,119
274,9816658.0,248352000.0,ROTTERDAM,2023-03-29 11:38:22+00:00,2023-03-29 12:28:55+00:00,"[4.18029, 51.94091]","[4.19519, 51.93326]",CALANDK HBR P 79,CALANDK PALEN 82,CKVTTI,...,51.94091,4.19519,51.93326,205770000,5.6,97.0,16.0,tanker,7,119
286,9520572.0,304068000.0,FAIRPLAY-27,2022-09-19 04:50:56+00:00,2022-09-19 05:41:19+00:00,"[4.03595, 51.96296]","[4.03538, 51.96287]",EUROPAH NESTE NZ,EUROPAH NESTE ZZ,EURO,...,51.96296,4.03538,51.96287,205772000,4.5,98.0,16.0,tanker,7,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16346,9765043.0,210176000.0,FAIRPLAY 28,2023-04-13 03:28:30+00:00,2023-04-13 04:29:32+00:00,"[4.07025, 51.9505]","[4.05641, 51.96287]",AMAZONEH ECT DDE,EUROPAH FWP 2,AMAZ,...,51.95050,4.05641,51.96287,636092635,6.2,133.0,18.0,cargo,0,154
16349,9365116.0,244616000.0,FAIRPLAY III,2023-04-20 14:32:01+00:00,2023-04-20 15:13:10+00:00,"[4.05867, 51.96606]","[4.00287, 51.95478]",EUROPAH FWP 1,AMALIAH APMT DSQ,YANG,...,51.96606,4.00287,51.95478,636092635,6.2,133.0,18.0,cargo,0,73
16388,9402433.0,249532000.0,VB SCHELDE,2023-03-13 00:25:20+00:00,2023-03-13 02:22:04+00:00,"[4.06329, 51.94934]","[4.05847, 51.94879]",AMAZONEH ECT DDE,AMAZONEH ECT DDE,AMAZ,...,51.94934,4.05847,51.94879,636092782,11.3,300.0,46.0,cargo,0,76
16505,9816359.0,248753000.0,BEAGLE,2022-09-30 20:56:58+00:00,2022-09-30 22:33:40+00:00,"[4.21438, 51.91707]","[4.1647, 51.9414]",7E PET VOPAK 2 WZ,5E PET GUNVOR WZ VP,7PET,...,51.91707,4.16470,51.94140,664445000,10.1,189.0,32.0,tanker,7,47


For the "from"

In [18]:
# Define categorical and numeric columns
#categorical_cols = ['vessel_type', 'vessel_callsign', 'vessel_subtype', 'vessel_name', 'navigation_status']
features = ['navigation_draught', 
            'length',	
            'width',  
            'vessel_type_Code', 
            'Incoming to']

X = main_db[features].values

# Assuming 'latitude' and 'longitude' are your target variables (y-values)
# Target variables 
y_from_lat = main_db['to_lat'].values
y_from_lon = main_db['to_long'].values


Linear Regression

In [29]:
# Split data into training and testing sets for latitude prediction
X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(X, y_from_lat, test_size=0.2, random_state=42)

# Initialize and train the linear regression model for latitude
linear_model_lat = LinearRegression()
linear_model_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = linear_model_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Linear Regression Mean Squared Error (Latitude): {mse_lat}")

# Split data into training and testing sets for longitude prediction
X_from_lon_train, X_from_lon_test, y_from_lon_train, y_from_lon_test = train_test_split(X, y_from_lon, test_size=0.2, random_state=42)

# Initialize and train the linear regression model for longitude
linear_model_lon = LinearRegression()
linear_model_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = linear_model_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Linear Regression Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
median = np.median(geodesic_distances)
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")
print(f"Mean Geodesic Distance: {median} meters")

# Create a map centered at a specific location (e.g., mean latitude and longitude of your data)
# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
a = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(a)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(a)

# Save the map as an HTML files
a.save('map_linear_regression.html')



Linear Regression Mean Squared Error (Latitude): 0.0007406431496214196
Linear Regression Mean Squared Error (Longitude): 0.013124108241050883
Geodesic Distance (meter) for Test Point 1: 3982.56348271158
Geodesic Distance (meter) for Test Point 2: 3247.0676141075787
Geodesic Distance (meter) for Test Point 3: 5646.617201258645
Geodesic Distance (meter) for Test Point 4: 1184.5734825011477
Geodesic Distance (meter) for Test Point 5: 4929.74931634927
Geodesic Distance (meter) for Test Point 6: 986.2948151879369
Geodesic Distance (meter) for Test Point 7: 9159.174681748811
Geodesic Distance (meter) for Test Point 8: 10804.98089276777
Geodesic Distance (meter) for Test Point 9: 9192.358894730238
Geodesic Distance (meter) for Test Point 10: 7529.61209451885
Geodesic Distance (meter) for Test Point 11: 4866.855449036639
Geodesic Distance (meter) for Test Point 12: 2663.879864981551
Geodesic Distance (meter) for Test Point 13: 18402.671117200967
Geodesic Distance (meter) for Test Point 14: 418

Geodesic Distance (meter) for Test Point 214: 18566.014456378012
Geodesic Distance (meter) for Test Point 215: 6850.159173659563
Geodesic Distance (meter) for Test Point 216: 8713.97814840037
Geodesic Distance (meter) for Test Point 217: 4738.613537422531
Geodesic Distance (meter) for Test Point 218: 1184.082513406717
Geodesic Distance (meter) for Test Point 219: 7327.636331270062
Geodesic Distance (meter) for Test Point 220: 14255.983620498157
Geodesic Distance (meter) for Test Point 221: 2687.6702196420692
Geodesic Distance (meter) for Test Point 222: 6772.531006284816
Geodesic Distance (meter) for Test Point 223: 5904.744069460666
Geodesic Distance (meter) for Test Point 224: 15180.821886886975
Geodesic Distance (meter) for Test Point 225: 5896.405968453508
Geodesic Distance (meter) for Test Point 226: 977.1668605183361
Geodesic Distance (meter) for Test Point 227: 5069.946972139812
Geodesic Distance (meter) for Test Point 228: 11782.336742134123
Geodesic Distance (meter) for Test P

In [20]:
from IPython.display import display

# Your previous code for creating the map
display(a)

Decision Tree

In [30]:
# Initialize and train the decision tree regression model for latitude
decision_tree_lat = DecisionTreeRegressor(random_state=42)
decision_tree_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = decision_tree_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Decision Tree Mean Squared Error (Latitude): {mse_lat}")

# Initialize and train the decision tree regression model for longitude
decision_tree_lon = DecisionTreeRegressor(random_state=42)
decision_tree_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = decision_tree_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Decision Tree Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

median = np.median(geodesic_distances)
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")
print(f"Mean Geodesic Distance: {median} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
b = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(b)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(b)

# Save the map as an HTML file
b.save('map_decision_tree.html')
display(b)



Decision Tree Mean Squared Error (Latitude): 3.414250289778967e-05
Decision Tree Mean Squared Error (Longitude): 0.0024747614321845766
Geodesic Distance (meter) for Test Point 1: 390.4172768368068
Geodesic Distance (meter) for Test Point 2: 2.344483094338968
Geodesic Distance (meter) for Test Point 3: 10177.159319925628
Geodesic Distance (meter) for Test Point 4: 128.35504195180064
Geodesic Distance (meter) for Test Point 5: 638.1943084673654
Geodesic Distance (meter) for Test Point 6: 0.0
Geodesic Distance (meter) for Test Point 7: 2.2253292444736017
Geodesic Distance (meter) for Test Point 8: 226.62902805390183
Geodesic Distance (meter) for Test Point 9: 0.0
Geodesic Distance (meter) for Test Point 10: 2.3435392550601364
Geodesic Distance (meter) for Test Point 11: 143.66540900684763
Geodesic Distance (meter) for Test Point 12: 0.0
Geodesic Distance (meter) for Test Point 13: 0.0
Geodesic Distance (meter) for Test Point 14: 6.440471515629182
Geodesic Distance (meter) for Test Point 1

Random Forest Regression Model

In [26]:
# Initialize and train the random forest regression model for latitude
random_forest_lat = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = random_forest_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Random Forest Mean Squared Error (Latitude): {mse_lat}")

# Initialize and train the random forest regression model for longitude
random_forest_lon = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = random_forest_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Random Forest Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
median = np.median(geodesic_distances)
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")
print(f"Mean Geodesic Distance: {median} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
c = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(c)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(c)

# Save the map as an HTML file
c.save('map_random_forest.html')
display(c)



Random Forest Mean Squared Error (Latitude): 4.8476242366220946e-05
Random Forest Mean Squared Error (Longitude): 0.0007407586617415109
Geodesic Distance (meter) for Test Point 1: 305.7950624417227
Geodesic Distance (meter) for Test Point 2: 44.732886861883195
Geodesic Distance (meter) for Test Point 3: 8862.13474951595
Geodesic Distance (meter) for Test Point 4: 139.47643635928372
Geodesic Distance (meter) for Test Point 5: 662.0452384035028
Geodesic Distance (meter) for Test Point 6: 8.110182609869621
Geodesic Distance (meter) for Test Point 7: 85.88338997659363
Geodesic Distance (meter) for Test Point 8: 232.18242740589665
Geodesic Distance (meter) for Test Point 9: 427.3349923398927
Geodesic Distance (meter) for Test Point 10: 162.92729152978714
Geodesic Distance (meter) for Test Point 11: 301.9036471369287
Geodesic Distance (meter) for Test Point 12: 38.43177148791927
Geodesic Distance (meter) for Test Point 13: 488.84467692039726
Geodesic Distance (meter) for Test Point 14: 6.336

Gradient Boosting Regression Model

In [27]:
# Initialize and train the gradient boosting regression model for latitude
gradient_boosting_lat = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradient_boosting_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = gradient_boosting_lat.predict(X_from_lat_test)

# Initialize and train the gradient boosting regression model for longitude
gradient_boosting_lon = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradient_boosting_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = gradient_boosting_lon.predict(X_from_lon_test)

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
median = np.median(geodesic_distances)
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")
print(f"Mean Geodesic Distance: {median} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
d = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):  # You can change this to display more or fewer points
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(d)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(d)

# Save the map as an HTML file
d.save('map_gradient_boosting.html')
display(d)


Geodesic Distance (meter) for Test Point 1: 1538.8314711319315
Geodesic Distance (meter) for Test Point 2: 405.0390124327676
Geodesic Distance (meter) for Test Point 3: 7641.507507070657
Geodesic Distance (meter) for Test Point 4: 395.77884968586636
Geodesic Distance (meter) for Test Point 5: 1264.880709971894
Geodesic Distance (meter) for Test Point 6: 1919.4163342431286
Geodesic Distance (meter) for Test Point 7: 465.2072911659633
Geodesic Distance (meter) for Test Point 8: 1328.8706711855164
Geodesic Distance (meter) for Test Point 9: 1634.4164481152259
Geodesic Distance (meter) for Test Point 10: 832.9136385582195
Geodesic Distance (meter) for Test Point 11: 1132.7501133532653
Geodesic Distance (meter) for Test Point 12: 507.7039851837475
Geodesic Distance (meter) for Test Point 13: 6309.4935314256345
Geodesic Distance (meter) for Test Point 14: 818.5465342468239
Geodesic Distance (meter) for Test Point 15: 307.5291237197274
Geodesic Distance (meter) for Test Point 16: 257.79218131

In [28]:
from xgboost import XGBRegressor

# Initialize and train the XGBoost regression model for latitude
xgb_lat = XGBRegressor(n_estimators=100, random_state=42)
xgb_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = xgb_lat.predict(X_from_lat_test)

# Initialize and train the XGBoost regression model for longitude
xgb_lon = XGBRegressor(n_estimators=100, random_state=42)
xgb_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = xgb_lon.predict(X_from_lon_test)

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average geodesic distance
median = np.median(geodesic_distances)
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")
print(f"Mean Geodesic Distance: {median} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
xgb_map = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):  # You can change this to display more or fewer points
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(xgb_map)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(xgb_map)

# Save the map as an HTML file
xgb_map.save('map_xgboost.html')
display(xgb_map)


Geodesic Distance (meter) for Test Point 1: 142.8095390190471
Geodesic Distance (meter) for Test Point 2: 62.913009659170434
Geodesic Distance (meter) for Test Point 3: 10184.121249672691
Geodesic Distance (meter) for Test Point 4: 129.23607204619606
Geodesic Distance (meter) for Test Point 5: 691.7277335827497
Geodesic Distance (meter) for Test Point 6: 34.13254710108508
Geodesic Distance (meter) for Test Point 7: 72.79521670564179
Geodesic Distance (meter) for Test Point 8: 2248.5786482747208
Geodesic Distance (meter) for Test Point 9: 261.12088654906364
Geodesic Distance (meter) for Test Point 10: 153.57996907854456
Geodesic Distance (meter) for Test Point 11: 480.41548797835964
Geodesic Distance (meter) for Test Point 12: 223.04837961360383
Geodesic Distance (meter) for Test Point 13: 264.6005169541562
Geodesic Distance (meter) for Test Point 14: 62.72875968074119
Geodesic Distance (meter) for Test Point 15: 47.23773689110752
Geodesic Distance (meter) for Test Point 16: 86.74881086

In [31]:
#SVM adapted for predictions
from sklearn.svm import SVR
import folium

# Initialize and train the SVR model for latitude
svr_lat = SVR(kernel='rbf')
svr_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = svr_lat.predict(X_from_lat_test)

# Initialize and train the SVR model for longitude
svr_lon = SVR(kernel='rbf')
svr_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = svr_lon.predict(X_from_lon_test)

# Calculate geodesic distance between true and predicted coordinates in meters
geodesic_distances = []
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    geodesic_distances.append(distance)
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Calculate average and median geodesic distances
median = np.median(geodesic_distances)
total_distance = np.sum(geodesic_distances)
average_distance = total_distance / len(geodesic_distances)
print(f"Average Geodesic Distance: {average_distance} meters")
print(f"Median Geodesic Distance: {median} meters")

# Define the latitude and longitude to center the map
center_latitude = 51.9775
center_longitude = 4.1331

# Create a map centered at the specified location
map_center = [center_latitude, center_longitude]
svr_map = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(1):  # You can change this to display more or fewer points
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(svr_map)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(svr_map)

# Save the map as an HTML file
svr_map.save('map_svr.html')
display(svr_map)


Geodesic Distance (meter) for Test Point 1: 3824.3690105089277
Geodesic Distance (meter) for Test Point 2: 2990.656558934592
Geodesic Distance (meter) for Test Point 3: 12871.175303842369
Geodesic Distance (meter) for Test Point 4: 6198.382404467937
Geodesic Distance (meter) for Test Point 5: 10817.183919012125
Geodesic Distance (meter) for Test Point 6: 2165.876606890731
Geodesic Distance (meter) for Test Point 7: 9578.335791474721
Geodesic Distance (meter) for Test Point 8: 1890.4602811244974
Geodesic Distance (meter) for Test Point 9: 4291.904394548132
Geodesic Distance (meter) for Test Point 10: 3661.5080170383853
Geodesic Distance (meter) for Test Point 11: 3824.794552524569
Geodesic Distance (meter) for Test Point 12: 7925.106028368526
Geodesic Distance (meter) for Test Point 13: 17956.603627829092
Geodesic Distance (meter) for Test Point 14: 4269.538637204852
Geodesic Distance (meter) for Test Point 15: 1619.3924334877966
Geodesic Distance (meter) for Test Point 16: 4508.0400193